In [10]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('Processed_data.csv')
df.head()

,STATE,IBGE_RES_POP,IBGE_RES_POP_BRAS,IBGE_DU,IBGE_DU_URBAN,IBGE_DU_RURAL,IBGE_POP,IBGE_1,IBGE_1-4,IBGE_5-9,...,Pu_Agencies,Pr_Bank,Pu_Bank,Cars,Motorcycles,Wheeled_tractor,UBER,MAC,WAL-MART,POST_OFFICES
0,16,5098,5098,1536,1100,436,3594,46,198,265,...,0,0,0,282,1185,0,0,0,0,1
1,10,2709,2706,875,364,511,1099,13,40,69,...,0,0,0,479,332,0,0,0,0,1
2,14,5239,5239,1567,639,928,2062,40,132,162,...,0,0,0,297,274,0,0,0,0,1
3,14,2366,2366,676,353,323,1231,22,113,119,...,0,0,0,198,104,0,0,0,0,1
4,9,11063,11063,2632,1118,1515,3154,31,204,316,...,0,0,0,196,1008,0,0,0,0,1


In [11]:
# Split the data into features and target
X = df.drop(columns=['IDHM'])
y = df['IDHM']

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split dados

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalização de Escalas 

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import ExtraTreeRegressor


# Definir o grid de hiperparâmetros 
param_grid = {
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [20, 30, 40],
    'min_samples_leaf': [1, 5, 10],
    'max_features': [None, 'sqrt', 'log2'],  
    'random_state': [66]  
}

# Configurar o GridSearch
grid_search = GridSearchCV(estimator=ExtraTreeRegressor(), param_grid=param_grid, cv=7, scoring='neg_mean_squared_error')

# Treinar e buscar os melhores parâmetros
grid_search.fit(X_train, y_train)

# Exibir os melhores hiperparâmetros
print("Melhores hiperparâmetros:", grid_search.best_params_)


Melhores hiperparâmetros: {'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 5, 'min_samples_split': 20, 'random_state': 66}


In [17]:
from sklearn.tree import ExtraTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

best_params = grid_search.best_params_

# Treinar o modelo 
et_model = ExtraTreeRegressor(
    random_state=57,
    max_depth=best_params['max_depth'],
    max_features=best_params['max_features'],
    min_samples_leaf=best_params['min_samples_leaf'],
    min_samples_split=best_params['min_samples_split']
    )

et_model.fit(X_train_scaled, y_train)

y_pred_et = et_model.predict(X_test_scaled)

# Avaliação do modelo
mse_et = mean_squared_error(y_test, y_pred_et)
r2_et = r2_score(y_test, y_pred_et)
print("ExtraTree - MSE:", mse_et, "R²:", r2_et)

ExtraTree - MSE: 0.0011623621776641435 R²: 0.4381208948623161
